In [1]:
import os
import csv
import time
import datetime
import random
import json
import threading

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [63]:
# 配置参数

class TrainingConfig(object):
    epoches = 5
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    epsilon = 5
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self.indexFreqs = []  # 统计词空间中的词在出现在多少个review中
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 得到逆词频
        self._getWordIndexFreq(vocab, reviews)
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _getWordIndexFreq(self, vocab, reviews):
        """
        统计词汇空间中各个词出现在多少个文本中
        """
        reviewDicts = [dict(zip(review, range(len(review)))) for review in reviews]
        indexFreqs = [0] * len(vocab)
        for word in vocab:
            count = 0
            for review in reviewDicts:
                if word in review:
                    count += 1
            indexFreqs[self._wordToIndex[word]] = count
        
        self.indexFreqs = indexFreqs
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
print(data.wordEmbedding.shape)

(31983, 200)


In [6]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [59]:
# 构建模型
class AdversarialLSTM(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding, indexFreqs):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.config = config
        
        # 根据词的频率计算权重
        indexFreqs[0], indexFreqs[1] = 20000, 10000
        weights = tf.cast(tf.reshape(indexFreqs / tf.reduce_sum(indexFreqs), [1, len(indexFreqs)]), dtype=tf.float32)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用词频计算新的词嵌入矩阵
            normWordEmbedding = self._normalize(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), weights)
            
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(normWordEmbedding, self.inputX)
            
         # 计算二元交叉熵损失 
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)
        
        with tf.name_scope("perturLoss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)
        
        self.loss = loss + perturLoss
            
    def _Bi_LSTMAttention(self, embeddedWords):
        """
        Bi-LSTM + Attention 的模型结构
        """
        
        config = self.config
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)

            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm, embeddedWords, dtype=tf.float32)
        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self._attention(H)
            outputSize = config.model.hiddenSizes[-1]
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            
        return predictions
    
    def _attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes[-1]
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output
    
    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        
        mean = tf.matmul(weights, wordEmbedding)
        print(mean)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)
        
        var = tf.matmul(weights, powWordEmbedding)
        print(var)
        stddev = tf.sqrt(1e-6 + var)
        
        return (wordEmbedding - mean) / stddev
    
    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, self.config.model.epsilon)
        return embedded + perturb
    
    def _scaleL2(self, x, norm_length):
        # shape(x) = (batch, num_timesteps, d)
        # Divide x by max(abs(x)) for a numerically stable L2 norm.
        # 2norm(x) = a * 2norm(x/a)
        # Scale over the full sequence, dims (1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(
            tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [8]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [64]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding
indexFreqs = data.indexFreqs

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = AdversarialLSTM(config, wordEmbedding, indexFreqs)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

         builder.save()

Tensor("embedding/MatMul:0", shape=(1, 200), dtype=float32)
Tensor("embedding/MatMul_1:0", shape=(1, 200), dtype=float32)
INFO:tensorflow:Summary name Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/hist is illegal; using Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/sparsity is illegal; using Bi-LSTM/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/sparsity instead.
INFO:t

2018-12-29T11:25:54.810604, step: 41, loss: 1.1981995105743408, acc: 0.7812, auc: 0.9126, precision: 0.9091, recall: 0.625
2018-12-29T11:25:55.524222, step: 42, loss: 1.3127529621124268, acc: 0.7031, auc: 0.8044, precision: 0.7872, recall: 0.5692
2018-12-29T11:25:56.195082, step: 43, loss: 1.2615294456481934, acc: 0.6797, auc: 0.8284, precision: 0.7949, recall: 0.4844
2018-12-29T11:25:56.916487, step: 44, loss: 1.2160320281982422, acc: 0.7266, auc: 0.8519, precision: 0.9286, recall: 0.4407
2018-12-29T11:25:57.727582, step: 45, loss: 1.1920576095581055, acc: 0.7969, auc: 0.9079, precision: 0.8824, recall: 0.5769
2018-12-29T11:25:58.464835, step: 46, loss: 1.3102998733520508, acc: 0.6328, auc: 0.8701, precision: 1.0, recall: 0.3896
2018-12-29T11:25:59.175739, step: 47, loss: 1.2885699272155762, acc: 0.6328, auc: 0.8441, precision: 0.8966, recall: 0.3714
2018-12-29T11:25:59.799921, step: 48, loss: 1.2007193565368652, acc: 0.7344, auc: 0.8955, precision: 0.9459, recall: 0.5224
2018-12-29T1

2018-12-29T11:27:10.163832, step: 107, loss: 1.06497061252594, acc: 0.8359, auc: 0.9115, precision: 0.8571, recall: 0.8182
2018-12-29T11:27:10.880213, step: 108, loss: 1.1167004108428955, acc: 0.8594, auc: 0.9043, precision: 0.8947, recall: 0.8095
2018-12-29T11:27:11.534325, step: 109, loss: 0.9497590065002441, acc: 0.8828, auc: 0.9611, precision: 0.9123, recall: 0.8387
2018-12-29T11:27:12.216082, step: 110, loss: 1.0824297666549683, acc: 0.7969, auc: 0.9091, precision: 0.8308, recall: 0.7826
2018-12-29T11:27:12.954587, step: 111, loss: 0.9668493866920471, acc: 0.8906, auc: 0.9548, precision: 0.9048, recall: 0.8769
2018-12-29T11:27:13.691028, step: 112, loss: 1.1021671295166016, acc: 0.8125, auc: 0.9122, precision: 0.7458, recall: 0.8302
2018-12-29T11:27:14.383268, step: 113, loss: 0.970019519329071, acc: 0.9062, auc: 0.9567, precision: 0.9455, recall: 0.8525
2018-12-29T11:27:15.135760, step: 114, loss: 1.0011255741119385, acc: 0.8594, auc: 0.9244, precision: 0.9219, recall: 0.8194
201

2018-12-29T11:27:57.455440, step: 173, loss: 0.8252881765365601, acc: 0.8516, auc: 0.918, precision: 0.85, recall: 0.8361
2018-12-29T11:27:58.188181, step: 174, loss: 0.9199938178062439, acc: 0.8359, auc: 0.925, precision: 0.9062, recall: 0.7945
2018-12-29T11:27:58.929226, step: 175, loss: 0.7192668914794922, acc: 0.9062, auc: 0.9633, precision: 0.9074, recall: 0.875
2018-12-29T11:27:59.627136, step: 176, loss: 0.8197203874588013, acc: 0.8125, auc: 0.9225, precision: 0.9375, recall: 0.6818
2018-12-29T11:28:00.295595, step: 177, loss: 0.7617433071136475, acc: 0.8438, auc: 0.9117, precision: 0.8475, recall: 0.8197
2018-12-29T11:28:01.027652, step: 178, loss: 0.8304474949836731, acc: 0.8438, auc: 0.9573, precision: 0.8824, recall: 0.7627
2018-12-29T11:28:01.710981, step: 179, loss: 0.738858699798584, acc: 0.8281, auc: 0.9479, precision: 0.8679, recall: 0.7541
2018-12-29T11:28:02.339048, step: 180, loss: 0.8300277590751648, acc: 0.8281, auc: 0.9396, precision: 0.8627, recall: 0.7458
2018-1

2018-12-29T11:29:10.947932, step: 238, loss: 0.45562341809272766, acc: 0.8359, auc: 0.9331, precision: 0.8772, recall: 0.7812
2018-12-29T11:29:11.659636, step: 239, loss: 0.4219127595424652, acc: 0.8672, auc: 0.9522, precision: 0.8909, recall: 0.8167
2018-12-29T11:29:12.393164, step: 240, loss: 0.5501759052276611, acc: 0.8594, auc: 0.937, precision: 0.898, recall: 0.7719
2018-12-29T11:29:13.134779, step: 241, loss: 0.5582970380783081, acc: 0.8516, auc: 0.9399, precision: 0.9107, recall: 0.7846
2018-12-29T11:29:13.823727, step: 242, loss: 0.4210389256477356, acc: 0.8984, auc: 0.9759, precision: 0.9216, recall: 0.8393
2018-12-29T11:29:14.504294, step: 243, loss: 0.599084198474884, acc: 0.8359, auc: 0.9339, precision: 0.9423, recall: 0.7313
2018-12-29T11:29:15.187181, step: 244, loss: 0.4720156192779541, acc: 0.8672, auc: 0.959, precision: 0.9273, recall: 0.7969
2018-12-29T11:29:15.844715, step: 245, loss: 0.49234265089035034, acc: 0.8828, auc: 0.9407, precision: 0.9077, recall: 0.8676
20

...

2018-12-29T11:33:12.095296, step: 500, loss: 0.28256040811538696, acc: 0.8984, auc: 0.9604, precision: 0.9322, recall: 0.8594

Evaluation:
2018-12-29T11:33:39.057464, step: 500, loss: 0.33821701297634527, acc: 0.8729394736842103, auc: 0.9466710526315791, precision: 0.8974368421052632, recall: 0.8455631578947368
2018-12-29T11:33:39.703166, step: 501, loss: 0.2845889925956726, acc: 0.8906, auc: 0.9618, precision: 0.9259, recall: 0.8333
2018-12-29T11:33:40.427641, step: 502, loss: 0.34236016869544983, acc: 0.875, auc: 0.9441, precision: 0.9286, recall: 0.8125
2018-12-29T11:33:41.122354, step: 503, loss: 0.3332943320274353, acc: 0.8906, auc: 0.9577, precision: 0.8704, recall: 0.8704
2018-12-29T11:33:41.873078, step: 504, loss: 0.32223227620124817, acc: 0.8672, auc: 0.9465, precision: 0.8889, recall: 0.8136
2018-12-29T11:33:42.583433, step: 505, loss: 0.3783959448337555, acc: 0.8828, auc: 0.9422, precision: 0.898, recall: 0.8148
2018-12-29T11:33:43.250335, step: 506, loss: 0.242227703332901

2018-12-29T11:34:26.151227, step: 565, loss: 0.29564550518989563, acc: 0.875, auc: 0.9492, precision: 0.9298, recall: 0.8154
2018-12-29T11:34:26.888838, step: 566, loss: 0.3555968701839447, acc: 0.8672, auc: 0.9572, precision: 0.9583, recall: 0.7541
2018-12-29T11:34:27.616099, step: 567, loss: 0.25651612877845764, acc: 0.8984, auc: 0.9741, precision: 0.9038, recall: 0.8545
2018-12-29T11:34:28.309029, step: 568, loss: 0.2928429841995239, acc: 0.9062, auc: 0.9649, precision: 0.9692, recall: 0.863
2018-12-29T11:34:29.036520, step: 569, loss: 0.3691571354866028, acc: 0.8594, auc: 0.9261, precision: 0.9048, recall: 0.8261
2018-12-29T11:34:29.690702, step: 570, loss: 0.2767903506755829, acc: 0.9062, auc: 0.9777, precision: 0.9016, recall: 0.9016
2018-12-29T11:34:30.432824, step: 571, loss: 0.3685939311981201, acc: 0.875, auc: 0.9468, precision: 0.9, recall: 0.8438
2018-12-29T11:34:31.199999, step: 572, loss: 0.2842484712600708, acc: 0.8672, auc: 0.9627, precision: 0.9057, recall: 0.8
2018-12

2018-12-29T11:35:40.376526, step: 630, loss: 0.2638986110687256, acc: 0.8906, auc: 0.9623, precision: 0.9273, recall: 0.8361
2018-12-29T11:35:41.093155, step: 631, loss: 0.2970955967903137, acc: 0.9062, auc: 0.9459, precision: 0.9275, recall: 0.9014
2018-12-29T11:35:41.828061, step: 632, loss: 0.3524641692638397, acc: 0.8906, auc: 0.9436, precision: 0.9423, recall: 0.8167
2018-12-29T11:35:42.524684, step: 633, loss: 0.19946140050888062, acc: 0.8906, auc: 0.9903, precision: 0.9848, recall: 0.8333
2018-12-29T11:35:43.216848, step: 634, loss: 0.2845154106616974, acc: 0.875, auc: 0.9624, precision: 0.9107, recall: 0.8226
2018-12-29T11:35:43.926645, step: 635, loss: 0.30302852392196655, acc: 0.8594, auc: 0.9519, precision: 0.8548, recall: 0.8548
2018-12-29T11:35:44.626818, step: 636, loss: 0.274955689907074, acc: 0.8516, auc: 0.9582, precision: 0.9138, recall: 0.791
2018-12-29T11:35:45.327882, step: 637, loss: 0.3288918137550354, acc: 0.8828, auc: 0.9618, precision: 0.9104, recall: 0.8714
2

2018-12-29T11:36:27.564139, step: 696, loss: 0.23871512711048126, acc: 0.8828, auc: 0.9726, precision: 0.9444, recall: 0.8095
2018-12-29T11:36:28.271945, step: 697, loss: 0.30430859327316284, acc: 0.9141, auc: 0.9643, precision: 0.918, recall: 0.9032
2018-12-29T11:36:28.947855, step: 698, loss: 0.37159526348114014, acc: 0.8359, auc: 0.9591, precision: 1.0, recall: 0.7
2018-12-29T11:36:29.679913, step: 699, loss: 0.23637986183166504, acc: 0.8828, auc: 0.9774, precision: 0.9846, recall: 0.8205
2018-12-29T11:36:30.346117, step: 700, loss: 0.232436865568161, acc: 0.9141, auc: 0.9769, precision: 0.8889, recall: 0.9057

Evaluation:
2018-12-29T11:36:57.695813, step: 700, loss: 0.3329193944993772, acc: 0.8754105263157891, auc: 0.9468947368421052, precision: 0.8791763157894735, recall: 0.8737921052631578
2018-12-29T11:36:58.423105, step: 701, loss: 0.20016704499721527, acc: 0.9375, auc: 0.986, precision: 0.931, recall: 0.931
2018-12-29T11:36:59.161040, step: 702, loss: 0.23768414556980133, acc:

2018-12-29T11:37:41.400369, step: 761, loss: 0.21439702808856964, acc: 0.9141, auc: 0.9788, precision: 0.9403, recall: 0.9
2018-12-29T11:37:42.183186, step: 762, loss: 0.26376229524612427, acc: 0.9141, auc: 0.9844, precision: 0.9655, recall: 0.8615
2018-12-29T11:37:42.930009, step: 763, loss: 0.37042880058288574, acc: 0.8672, auc: 0.9471, precision: 0.9, recall: 0.7895
2018-12-29T11:37:43.639033, step: 764, loss: 0.26977697014808655, acc: 0.9062, auc: 0.957, precision: 0.9091, recall: 0.8772
2018-12-29T11:37:44.337299, step: 765, loss: 0.3444329500198364, acc: 0.8906, auc: 0.9522, precision: 0.9516, recall: 0.8429
2018-12-29T11:37:45.079281, step: 766, loss: 0.33014151453971863, acc: 0.875, auc: 0.9428, precision: 0.9167, recall: 0.8333
2018-12-29T11:37:45.804334, step: 767, loss: 0.23364895582199097, acc: 0.9062, auc: 0.971, precision: 0.9254, recall: 0.8986
2018-12-29T11:37:46.515896, step: 768, loss: 0.2611049711704254, acc: 0.9297, auc: 0.9669, precision: 0.9259, recall: 0.9091
201